# Flower Image Data - Preprocessing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import glob
import mahotas as mt

# Shape (Hu Moments)

In [ ]:
# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    
    feature = np.mean(feature)
    return feature

# Haralick Texture

In [ ]:
# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # compute the haralick texture feature vector
    haralick = mt.features.haralick(gray).mean(axis=0)
    
    # return the result
    haralick = np.mean(haralick)
    return haralick

# Color Histogram

In [ ]:
# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    chans = cv2.split(image)
    colors = ("b", "g", "r")

    # loop over the image channels
    for (chan, color) in zip(chans, colors):
        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])

    hist = np.mean(hist)
    return hist

# Processing all Images using Haralic Texture

In [ ]:
datadir="File path"
CATEGORIES = [ "bluebell", "daffodil","iris","snow","tigerlily"]

# empty list to hold feature vectors and train labels
train_features = []
train_labels = []
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(datadir, category)   
        
        class_num = CATEGORIES.index(category)    
        for img in os.listdir(path):
            # exception is used to pass broken images.
            try:
                image = cv2.imread(os.path.join(path,img))

                # extract haralick texture from the image
                feature1 = fd_hu_moments(image)
                feature2 = fd_haralick(image)
                feature3 = fd_histogram(image)

                # append the feature vector and label
                training_data.append([[feature1, feature2, feature3], class_num])
                
            except Exception as e:
                pass
            
create_training_data()   

In [ ]:
print(training_data)

In [ ]:
import random

In [ ]:
random.shuffle(training_data)

In [ ]:
# to check whether data is randomized. 
for check in training_data[:10]:
    print(check[0])    
    print(check[1])   

In [ ]:
for features, label in training_data:
    train_features.append(features)
    train_labels.append(label)
    
print(train_features[0])       
print(train_labels[0])

# Training Flower Data using Decision Tree

In [ ]:
import sklearn.tree as tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import graphviz
import matplotlib.image as mpimg

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_features,train_labels,test_size=0.25,random_state=1)

In [ ]:
X_train = np.array(X_train).reshape(-1, 3)      
decision_tree_classifier=DecisionTreeClassifier()
dtree = decision_tree_classifier.fit(X_train,y_train)
print(dtree)

In [ ]:
fn = ["shape","texture","color"]
tn = ["bluebell","daffodil","iris","snow","tigerlily"]

dot = tree.export_graphviz(dtree, out_file=None, feature_names=fn, class_names=tn, filled=True, rounded=True, special_characters=True)
 
graph = graphviz.Source(dot)
graph.format = 'png'
graph.render('decision_tree', view=True)

In [ ]:
X_test = np.array(X_test).reshape(-1, 3)     

decision_tree_classifier.score(X_test,y_test)

# Testing a sample

In [ ]:
flower = ["bluebell", "daffodil","iris","snow","tigerlily"]
sample = "Image path"
image = cv2.imread(sample)

feature1 = fd_hu_moments(image)
feature2 = fd_haralick(image)
feature3 = fd_histogram(image)
fts = [feature1, feature2, feature3]

img = mpimg.imread(sample)
plt.axis("off")
plt.imshow(img)
plt.show()

In [ ]:
features = np.array(fts).reshape(-1, 3)         

prediction = dtree.predict(features)
print("Predicted Label :",prediction)
print("Predicted Class :",flower[prediction[0]])